In [13]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy
import pandas as pd
import os
import random


print("-" *40)
print("연습문제 :네이버 영화 리뷰 정보 수집하기 ")
print("=" *40)
print("\n")

title = input('   1. 크롤링 할 영화의 제목을 입력하세요. : ')
url = "https://movie.naver.com/"

cnt = int(input('   2. 크롤링 할 리뷰 건수는 몇건입니까? : '))
page_cnt = math.ceil(cnt/10)

f_dir = input('   3. 파일을 저장할 폴더명만 쓰세요(예:C:\py_temp\) : ')
print("-" *40)

#파일 이름 지정
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
os.makedirs(f_dir+s+'-'+title)
os.chdir(f_dir+s+'-'+title)

ft_name=f_dir+s+'-'+title+'\\'+s+'-'+title+'.txt'
fc_name=f_dir+s+'-'+title+'\\'+s+'-'+title+'.csv'
fx_name=f_dir+s+'-'+title+'\\'+s+'-'+title+'.xls'


#크롬 드라이버 실행
s_time = time.time( ) #크롤링 시작시간

path = "C:\py_temp\chromedriver_win32\chromedriver.exe" # 컴퓨터 환경에 따라 변경해야함?
driver = webdriver.Chrome(path)
driver.get(url)
time.sleep(5)

element = driver.find_element_by_id("ipt_tx_srch")
element.send_keys(title)
driver.find_element_by_xpath('//*[@id="jSearchArea"]/div/button').click()
movies = driver.find_element_by_xpath('//*[@id="old_content"]/div[1]/span')

try :
    if movies == '(1-1 / 1건) ' :
        driver.find_element_by_xpath('//*[@id="old_content"]/ul[2]/li/p/a').click()
    else :
        driver.find_element_by_xpath('//*[@id="old_content"]/ul[2]/li[1]/dl/dt/a').click()
except :
    print("해당 영화가 존재하지 않습니다.")
    print("\n")


driver.find_element_by_link_text("평점").click()

#iframe 요소 가져오기!
driver.switch_to.frame('pointAfterListIframe')


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result= soup.find('div', class_='score_total').find('strong','total').find_all('em')
result2 = result[0].get_text()
#print(result[0])
result3 = result2.replace(",","")
result4 = re.search("\d+",result3)
search_cnt = int(result4.group())

print("=" *40)
if cnt > search_cnt :
    cnt = search_cnt

print("전체 검색 결과 건수 :",search_cnt,"건")
print("실제 최종 출력 건수",cnt)
print("\n")
print("크롤링 할 총 페이지수: ",page_cnt)
print("=" *40)

score2=[] #1. 별점
review2=[] #2. 리뷰내용
writer2=[] #3. 작성자
wdate2=[] #4. 작성일자
g_gogam2=[] #5.공감횟수
b_gogam2=[] #6.비공감횟수


count = 0         # 크롤링 건수 카운트 변수
click_count = 1   # 페이지 번호


score_result = soup.find('div', class_='score_result').find('ul')
slist = score_result.find_all('li')
    
while ( click_count <= page_cnt )  :
            

            for li in slist:

                count += 1
                
                f = open(ft_name, 'a' ,encoding='UTF-8')
            
                print("\n")
                print("총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다========" %(cnt,count))
                score = li.find('div', class_='star_score').find('em').get_text()
                #print("1.별점:","*"*float(score),": ", score)
                print("1.별점:","*"*int(score),": ", score)
                score2.append(score)
                f.write("\n")
                f.write("총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다========" %(cnt,count) + "\n")
                f.write("1.별점:"+score + "\n")
            
                #review = li.find('div', class_='score_reple').find('p').find('span', {'id':'_filtered_ment_0'}).get_text()
                review = li.find('div', class_='score_reple').find('p').get_text()
                print("2.리뷰내용:",review)
                f.write("2.리뷰내용:" + review + "\n")
                review2.append(review)
         
                dwlist = li.find('div',class_='score_reple').find_all('em')
                writer = dwlist[0].find('span').get_text()
                print("3.작성자:",writer)
                f.write("3.작성자:" + writer + "\n")
                writer2.append(writer)
                
                wdate = dwlist[1].text
                print('4.작성일자:',wdate)
                f.write("4.작성일자:" + wdate + "\n")
                wdate2.append(wdate)
            
                gogam = li.find('div', class_='btn_area').find_all('strong')
                g_gogam = gogam[0].text
                print('5.공감:',g_gogam)
                f.write("5.공감:" + g_gogam + "\n")
                g_gogam2.append(g_gogam)
                
                b_gogam = gogam[1].text
                print('6.비공감:', b_gogam)
                f.write("6.비공감:" + b_gogam + "\n")
                b_gogam2.append(b_gogam)
                print("\n")

                

                if count == cnt :
                    break
            
            time.sleep(random.randrange(1,2))  

            click_count += 1
            
            if click_count > page_cnt :
                break
            else :
                driver.find_element_by_link_text("%s" %click_count).click()

            time.sleep(random.randrange(1,2))  # 3-8 초 사이에 랜덤으로 시간 선택


#Step 11. xls 형태와 csv 형태로 저장하기
naver_movie = pd.DataFrame()
naver_movie['별점(평점)']=score2
naver_movie['리뷰내용']=review2
naver_movie['작성자']=writer2
naver_movie['작성일자']=wdate2
naver_movie['공감횟수']=g_gogam2
naver_movie['비공감횟수']=b_gogam2

# csv 형태로 저장하기
naver_movie.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
naver_movie.to_excel(fx_name ,index=True)

e_time = time.time( ) #크롤링 종료시간
t_time = e_time - s_time


print("\n") 
print("=" *40)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ft_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *40)


#1. 별점

#2. 리뷰내용

#3. 작성자

#4. 작성일자

#5.공감횟수

#6.비공감횟수

----------------------------------------
연습문제 :네이버 영화 리뷰 정보 수집하기 


   1. 크롤링 할 영화의 제목을 입력하세요. : 혼자 사는 사람들
   2. 크롤링 할 리뷰 건수는 몇건입니까? : 10
   3. 파일을 저장할 폴더명만 쓰세요(예:C:\py_temp\) : C:\py_temp\
----------------------------------------
전체 검색 결과 건수 : 140 건
실제 최종 출력 건수 10


크롤링 할 총 페이지수:  1


총 10 건 중 1 번째 리뷰 데이터를 수집합니다========
1.별점: ********** :  10
2.리뷰내용: 
관람객


														
															
															
																감독의 많은 고민이 느껴지는 영화입니다 대사가 많지 않음에도 주연 배우의 연기가 훌륭합니다 
															
														
														
													

3.작성자: oops****
4.작성일자: 2021.05.19 17:18
5.공감: 38
6.비공감: 5




총 10 건 중 2 번째 리뷰 데이터를 수집합니다========
1.별점: ********** :  10
2.리뷰내용: 
관람객


														
															
															
																별다른 기교를 부리지않았어도 연출력 뛰어나고 여주인공 연기도 섬세해서 몰입도 높은 영화였어요. 1인가구가 확산되는 요즘 시대에 한번쯤 볼만한 영화라 추천하고 싶네요. 
															
														
														
													

3.작성자: 윤정이(smba****)
4.작성일자: 2021.05.19 20:37
5.공감: 31
6.비공감: 2




총 10 건 중 3 번째 리뷰 데이터를 수집합니다========
1.별점

<ipython-input-13-bd7cba7308b9>:179: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  naver_movie.to_excel(fx_name ,index=True)
